# Streamline Modeling With Amazon SageMaker Studio and Amazon Experiments SDK

Modeling phase is a highly iterative process in Machine Learning projects, where Data Scientists experiment with various data pre-processing and feature engineering strategies, intertwined with different model architectures, which are then trained with disparate sets of hyperparameter values. This highly iterative process, with many moving parts, can, over time, manifest into a tremendous headache in terms of keeping track of all design decisions applied in each iteration and how the training and evaluation metrics of each iteration compare to the previous versions of the model.

This notebook walks you through an end-to-end example of how [Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-studio.html) can effectively leverage [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) to organize, track, visualize and compare our iterative work during development of a Keras model, trained to predict the age of an abalone, a sea snail, based on a set of features that describe it. While this example is specific to Keras, the same approach can be extended to other Machine Learning frameworks and algorithms. 

Amazon SageMaker Studio and Amazon SageMaker Experiments were unveiled at the [AWS Re:invent](https://aws.amazon.com/new/reinvent/), at the end of 2019:
* [SageMaker Studio Announcement](https://aws.amazon.com/about-aws/whats-new/2019/12/introducing-amazon-sagemaker-studio-the-first-integrated-development-environment-ide-for-machine-learning/?trk=ls_card)
* [SageMaker Experiments Announcement](https://aws.amazon.com/about-aws/whats-new/2019/12/introducing-amazon-sagemaker-experiments-organize-track-and-compare-your-machine-learning-training-experiments-on-amazon-sagemaker/?trk=ls_card)

In this walkthrough, we will explore how Amazon SageMaker Studio, and the [Experiments SDK](https://sagemaker-experiments.readthedocs.io/en/latest/), which has been [open-sourced](https://github.com/aws/sagemaker-experiments), can be utilized to experiment with a Keras model and track data preprocessing required to prepare data for the model's consumption.

Now, before we dive into hands-on exercise, let's first take a step back and discuss the building blocks of each Experiment and their referential relationships.
* **Experiment** - a Machine Learning problem that we want to solve. Each experiment consists of a collection of Trials. Note that the name of an experiment must be unique in a given region of a particular AWS account.
* **Trial** - an execution of a data-science workflow related to an experiment. Each Trial consists of several Trial Components. Note that the name of a trial must be unique in a given region of a particular AWS account.
* **Trial Component** - a stage in a given trial. For instance, as we will see in our example, we will create one Trial Component for data pre-preprocessing stage and one Trial Component for model training. Similarly, in other use cases, we can also have a Trial Component for data post-processing. Unlike Experiments and Trials, Trial Components do not have to be uniquely named as they tend to represent the typical and very common stages in an ML pipeline.
* **Tracker** - a mechanism that records various metadata about a particular Trial Component, including any Parameters, Inputs, Outputs, Artifacts and Metrics. When creating a Tracker, each Tracker is linked to a particular Training Component.

![](img/experiment_structure_t.png)

## Develop, Train, Optimize and Deploy Scikit-Learn Random Forest

> *This notebook should work well with the `Python 3 (Data Science)` kernel in SageMaker Studio, or the `conda_python3` kernel in SageMaker Notebook Instances*

In this notebook we show how to use Amazon SageMaker to develop, train, tune and deploy a Random Forest model based using the popular ML framework [Scikit-Learn](https://scikit-learn.org/stable/index.html).

The example uses the *Boston Housing dataset* (provided by Scikit-Learn) - more details of which can be found [here](https://scikit-learn.org/stable/datasets/index.html#boston-dataset).

To understand the code, you might also find it useful to refer to:

* The guide on [Using Scikit-Learn with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/using_sklearn.html)
* The API doc for [Scikit-Learn classes in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/sagemaker.sklearn.html)
* The [SageMaker reference for Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client) (The general AWS SDK for Python, including low-level bindings for SageMaker as well as many other AWS services)


In [2]:
!pip install --upgrade sagemaker-experiments

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 42 kB 99 kB/s              
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


## Setup libraries and environment


In [4]:
# Python Built-Ins:
from datetime import datetime
import tarfile

# External Dependencies:
import os
import boto3
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import sagemaker
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

project_path = 'experiments-sklearn'
processed_data_path = os.path.join('s3://',bucket,project_path,'data/processed/')
artifacts_path = os.path.join(project_path,'artifacts')
output_path = os.path.join('s3://',bucket,project_path,'output/')

print('Using bucket ' + bucket)


Using bucket sagemaker-us-east-1-349934754982


## Prepare data
We load a dataset from sklearn, split it and send it to S3

In [5]:
# we use the Boston housing dataset 
data = load_boston()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX['target'] = y_test

In [7]:
trainX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.09103,0.0,2.46,0.0,0.4880,7.155,92.2,2.7006,3.0,193.0,17.8,394.12,4.82,37.9
1,3.53501,0.0,19.58,1.0,0.8710,6.152,82.6,1.7455,5.0,403.0,14.7,88.01,15.02,15.6
2,0.03578,20.0,3.33,0.0,0.4429,7.820,64.5,4.6947,5.0,216.0,14.9,387.31,3.76,45.4
3,0.38735,0.0,25.65,0.0,0.5810,5.613,95.6,1.7572,2.0,188.0,19.1,359.29,27.26,15.7
4,0.06724,0.0,3.24,0.0,0.4600,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34,22.6


In [8]:
# create directories
! mkdir -p data
! mkdir -p source
! mkdir -p model

# save data as csv
trainX.to_csv('data/boston_train.csv')
testX.to_csv('data/boston_test.csv')

## Create a training script

The SageMaker Scikit-Learn Framework Container provides the basic runtime, and we as users specify the actual training steps to run as a script file (or even a folder of several, perhaps including a *requirements.txt* file).

The below code initializes a `.py` file from here in the notebook.

The same script can be used at training time (run as a script) and inference time (imported as a module) - So below we:

- Define some specific functions to override default inference behavior (e.g. `model_fn()`), and
- Enclose the training entry point in an `if __name__ == '__main__'` *guard clause* so it only executes when the module is run as a script.

You can find detailed guidance in the documentation on [Preparing a Scikit-Learn training script](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#prepare-a-scikit-learn-training-script) (for training) and the [SageMaker Scikit-Learn model server](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#sagemaker-scikit-learn-model-server) (for inference).

In [9]:
%%writefile source/sklearn_training_script.py
# Python Built-Ins:
import argparse
import os

# External Dependencies:
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == '__main__':

    #------------------------------- parsing input parameters (from command line)
    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # RandomForest hyperparameters
    parser.add_argument('--n_estimators', type=int, default=10)
    parser.add_argument('--min_samples_leaf', type=int, default=3)

    # Data, model, and output directories
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train_dir', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test_dir', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train_file', type=str, default='boston_train.csv')
    parser.add_argument('--test_file', type=str, default='boston_test.csv')
    parser.add_argument('--features', type=str)  # explicitly name which features to use
    parser.add_argument('--target_variable', type=str)  # explicitly name the column to be used as target

    args, _ = parser.parse_known_args()

    #------------------------------- data preparation
    print('reading data')
    train_df = pd.read_csv(os.path.join(args.train_dir, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test_dir, args.test_file))

    print('building training and testing datasets')
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target_variable]
    y_test = test_df[args.target_variable]

    #------------------------------- model training
    print('training model')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        n_jobs=-1)

    model.fit(X_train, y_train)

    #-------------------------------  model testing
    print('testing model')
    abs_err = np.abs(model.predict(X_test) - y_test)

    # percentile absolute errors
    for q in [10, 50, 90]:
        print('AE-at-' + str(q) + 'th-percentile: '
              + str(np.percentile(a=abs_err, q=q)))

    #------------------------------- save model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print('model saved at ' + path)


Writing source/sklearn_training_script.py


## Local training
Script arguments allows us to remove from the script any SageMaker-specific configuration, and run locally

In [ ]:
! python source/sklearn_training_script.py \
    --n_estimators 100 \
    --min_samples_leaf 3 \
    --model_dir 'model/' \
    --train_dir 'data/' \
    --test_dir 'data/' \
    --train_file 'boston_train.csv' \
    --test_file 'boston_test.csv' \
    --features 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT' \
    --target_variable 'target'

### Creating data input channels (copy to S3)

In [18]:
# send data to S3. SageMaker will take training data from s3
train_path_s3 = sess.upload_data(
    path='data/boston_train.csv',  # source
    bucket=bucket,
    key_prefix='sm101/sklearn'  # destination path in S3
)

test_path_s3 = sess.upload_data(
    path='data/boston_test.csv',  # source
    bucket=bucket,
    key_prefix='sm101/sklearn'  # destination path in S3
)

print('Train set URI:', train_path_s3)
print('Test set URI:', test_path_s3)

Train set URI: s3://sagemaker-us-east-1-349934754982/sm101/sklearn/boston_train.csv
Test set URI: s3://sagemaker-us-east-1-349934754982/sm101/sklearn/boston_test.csv


Now, let's utilize the Experiments SDK to create a Pre-processing Trial Component and keep track of what've accomplished so far. We start by creating an experiment. Remember that each experiment name must be unique within a given region of a particular AWS account.

Once we have our experiment instantiated, we will proceed with creating a Tracker to capture various details about the data pre-processing we completed so far.

In [25]:
from datetime import datetime
sm = boto3.client('sagemaker')
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')

boston_experiment = Experiment.create(
    experiment_name = 'predict-boston-' + ts,
    description = 'Predicting target var from boston dataset',
    sagemaker_boto_client=sm)


with Tracker.create(display_name='Pre-processing', sagemaker_boto_client=sm, artifact_bucket=bucket, artifact_prefix=artifacts_path) as tracker:
    tracker.log_parameters({
        'train_test_split': 0.75
    })
    tracker.log_input(name='raw data', media_type='s3/uri', value='sklearn.load_boston()')
    tracker.log_output(name='train data', media_type='s3/uri', value=train_path_s3)
    tracker.log_output(name='test data', media_type='s3/uri', value=test_path_s3)
    
processing_component = tracker.trial_component

Fantastic! We now have our experiment ready and we've already done our due diligence to capture our data pre-processing approach as a Trial Component. Next, let's dive into the modeling phase.

## Modeling with SageMaker Experiments
We are ready to define a few sets of hyperparameters that we want to experiment with and kick off our training jobs to run in parallel. Note that we can also easily capture custom metrics during training by configuring a regex pattern to match the desired values in the logs generated by the training job. In our case, for demonstration purposes, we will capture the median absolute error (`median-AE`) that the sklearn framework reports as the metric for our model.

Note that, in our example, we do not make any changes to our pre-processing strategy between different trials, so we will attach the same Pre-processing Trial Component that we create earlier, to each of the Trials we create in the next step.

In [26]:
# Define sets of hyperparameters, create a trial for each and kick of training jobs
hyperparameters_groups=[{
                        'n_estimators': 50,
                        'min_samples_leaf': 3,
                        'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
                        'target_variable': 'target',
                        },
                        {
                        'n_estimators': 100,
                        'min_samples_leaf': 3,
                        'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
                        'target_variable': 'target',
                        },
                        {
                        'n_estimators': 50,
                        'min_samples_leaf': 5,
                        'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
                        'target_variable': 'target',
                        },
                        {
                        'n_estimators': 100,
                        'min_samples_leaf': 5,
                        'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
                        'target_variable': 'target',
                        }]

In [27]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

for i,hp_set in enumerate(hyperparameters_groups):

    ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
    boston_trial = boston_experiment.create_trial(trial_name='boston-trial-' + str(i) + '-'+ ts)
    boston_trial.add_trial_component(processing_component)
    
    boston_estimator = SKLearn(entry_point='source/sklearn_training_script.py',
                                role=get_execution_role(),
                                instance_count=1,
                                instance_type='ml.m5.large',
                                framework_version='0.23-1',
                                base_job_name='rf-scikit',
                                hyperparameters=hp_set,
                                output_path = output_path,
                                metric_definitions=[
                                    { 'Name': 'median-AE', 'Regex': 'AE-at-50th-percentile: ([0-9.]+).*$' },
                                ],
                                max_run=20*60,  # Maximum allowed active runtime (in seconds)
                                use_spot_instances=True,  # Use spot instances to reduce cost
                                max_wait=30*60,  # Maximum clock time (including spot delays)
                            )
    


    job_name = 'boston-train-' + str(i) + '-'+ ts
    
    boston_estimator.fit({'train':train_path_s3, 'test': test_path_s3},
                          job_name=job_name,
                          wait=False,
                          experiment_config={
                                            'ExperimentName': boston_experiment.experiment_name,
                                            'TrialName': boston_trial.trial_name,
                                            'TrialComponentDisplayName': 'Training',
                                            })

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: boston-train-0-2022-03-01-17-28-22-900018
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: boston-train-1-2022-03-01-17-28-23-565612
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
I

Remember that the training job that we ran is very "light", due to the very small dataset. As such, running locally on the notebook instance results in a faster execution time, compared to SageMaker. SageMaker takes longer time to run the job because it has to provision the training infrastructure. Since this example training job not very resource-intensive, the infrastructure provisioning process adds more overhead, compared to the training job itself. 

In a real situation, where datasets are large, running on SageMaker can considerably speed up the execution process - and help us optimize costs, by keeping this interactive notebook environment modest and spinning up more powerful training job resources on-demand.

Note that this training job *did not run here on the notebook itself*. You'll be able to see the history in the [AWS Console for SageMaker - Training Jobs tab](https://console.aws.amazon.com/sagemaker/home?#/jobs).

> ℹ️ **Tip:** There's **no need to re-run** a training job if your notebook kernel restarts or the estimator state is lost for some other reason... You can just *attach* to a previous training job by name - for example:
>
> ```python
> estimator = SKLearn.attach('rf-scikit-2025-01-01-00-00-00-000')
> ```

## Exploration

Upon completion of the training jobs, we can, in just a few seconds, start visualizing how different variations of the model compare in terms of the metrics collected during model training. For instance, in just a few clicks, we can visualize how the loss has been decreasing by epoch for each variation of the model and very quickly observe the model architecture that is most effective in decreasing the loss.

![](img/visualize_loss.png)

You can create the same plot by following the five simple steps listed below:
1. Select the *SageMaker Experiments List* icon on the left sidebar.
2. Double-click on your experiment to open it and use Shift key on your keyboard to select all four trials.
3. Right-click on any of the highlighted trials and select *Open in trial component list*.
4. Use Shift key on your keyboard to select the four Trial Components representing the Training jobs and click on *Add chart* button.
5. Click on *New chart* and customize it to plot the collected metrics that you would like to analyze.

![](img/visualization_steps_line.gif)

Similarly, we can generate a scatter plot that helps us determine whether there is a relationship between the size of the first hidden layer in the network and the Mean Squared Logarithmic Error.

![](img/scatter_msle.png)
